In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import statistics
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
plt.style.use('seaborn')

In [3]:
'''
- check tensorflow and keras version
'''
print(f'Tensorflow GPU Version: {tf.__version__}')
print(f'Eager Execution is: {tf.executing_eagerly()}')
print(f'Keras Version: {tf.keras.__version__}')

Tensorflow GPU Version: 2.0.0
Eager Execution is: True
Keras Version: 2.2.4-tf


In [4]:
'''
- check GPU 
'''
var = tf.Variable([3,3])
if tf.test.is_gpu_available():
    print('Running on GPU')
else:
    print('Runing on CPU')

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Running on GPU


Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

# MLP model

In [10]:
'''
- MLP layer
'''
class MLP(tf.keras.layers.Layer):
    def __init__(self, units=3, activation=None, trainable=True, name=None, dtype=tf.float32, **kwargs):
        super(MLP, self).__init__( name=name, trainable = trainable, dtype=dtype, **kwargs)
        self.units = units
        self.__activation_name = activation
        self.activation = tf.keras.activations.get(activation)
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),initializer='random_normal',)
        self.b = self.add_weight(shape=(self.units,),initializer='random_normal')
        
    def get_config(self):
        config_dic ={
            'units':self.units,
            'activation': self.__activation_name, 
            'trainable_weights & bias':self.trainable_weights,
            'non-trainable_weights & bias':self.non_trainable_weights,
        }
        config = super(MLP,self).get_config()
        config.update(config_dic)
        return config
    
    def call(self, inputs, training = None):
        if tf.rank(inputs)==1:
            inputs = tf.keras.backend.expand_dims(inputs, axis=0)
        linear_combination = tf.matmul(inputs, self.w)+self.b 
        return self.activation(linear_combination)

# Multi-layer Perceptron Model

In [7]:
'''
- subclass model
'''
class MLP_Model(tf.keras.Model):
    def __init__(self, **kwargs):
        super(MLP_Model, self).__init__(self, **kwargs)
        self.mlp1 = MLP(256, activation='sigmoid')
        self.mlp2 = MLP(128, activation='sigmoid')
        self.mlp3 = MLP(10, acitivation='softmax')
        
    def call(self, inputs):
        opt = self.mlp1(inputs)
        opt = self.mlp2(opt)
        opt = self.mlp3(opt)
        return opt

# Logistic Regression Model

In [12]:
'''
- logistic regression
- perform binary classification with multi-labels
'''
class LogisticRegression(tf.keras.Model):
    def __init__(self,labels=1, **kwargs):
        super(LogisticRegression, self).__init__(self, **kwargs)
        self.label = labels
        self.lglayer = MLP(labels, activation='sigmoid')
    
    def call(self, inputs):
        opt= self.lglayer(inputs)
        return opt

In [13]:
lg_model = LogisticRegression(labels=3)

# Weighted KNN

In [89]:
class WKNN(tf.keras.Model):
    '''
    - weighted KNN algorithm refers to the paper published by Klaus Hechenbichler and Klaus Schliep in 2004
    - paper link: https://epub.ub.uni-muenchen.de/1769/1/paper_399.pdf
    - the paper provoides 3 similarity calculations and 8 weight transfer kernels, but in this model only basics are implemented
        similarity calculation: Euclidean Distance
        weight transfer calculation: Inversion Kernel
    - class arguments: 
        train_data: numpy array, tensor data, 
        train_labels: numpy array, tensor data, 
        k_value: must be an integer
    - call method: 
        
    '''
    def __init__(self, train_data, train_labels, k_value, **kwargs):
        super(WKNN, self).__init__(self, **kwargs)
        self._K = k_value
        self._data = tf.convert_to_tensor(train_data, dtype=tf.float32)
        self._label = tf.convert_to_tensor(train_labels)
        self._inputs = None
        self._clf_res = None
    
    @tf.function
    def __clf_predict(self, weights):
        # extract top kth weights and indices
        top_k_weights, top_k_indicies = tf.math.top_k(weights, self._K)
        # extract top kth labels 
        top_k_labels = tf.gather(self._label, top_k_indicies)
        # cast labels to float type and concatenate to the weights
        top_k_labels = tf.cast(tf.expand_dims(top_k_labels, axis=1), tf.float16)
        weight_label_matrix = tf.concat(top_k_weights, top_k_labels)
        # sort weight_label_matrix in ascending way based on the label order
        ascending_index = tf.argsort(weight_label_matrix[:,1], direction='ASCENDING')
        sorted_weight_label_matrix = tf.gather(weight_label_matrix, ascending_index)
        # get segment for labels and weights
        segment= tf.cast(sorted_weight_label_matrix[:,1],tf.int32)
        segment_weight = tf.expand_dims(sorted_weight_label_matrix[:,0], axis=1)
        # sum all weights in each label segment
        weight_sum = tf.math.segment_sum(segment_weight, segment)
        # max weight is the predict classification type
        return tf.argmax(weight_sum)
    
    @tf.function
    def __calc_similarity(self, transistion_matrix):
        similarity = tf.reduce_sum(tf.math.squared_difference(transition_matrix, self.__data, name='similarity'), axis=1)
        return similarity
    
    @tf.function
    def __inversion_kernel(self, similarity):
        zero_replacements = tf.math.multiply(tf.ones(shape=(similarity.shape)),0.1)
        similarity = tf.where(tf.math.not_equal(similarity, 0), similarity, zero_replacements)
        return tf.math.divide_no_nan(1, similarity)
    
    @tf.function
    def __wknn(self, row):
        # get a transistion matrix to expend 1*n to m*n, m is rows of training data
        trans_row = tf.ones(shape=(self._data.shape[0],1))
        row = tf.expand_dims(row, axis=0)
        transition_matrix = tf.matmul(trans_row, row)        
        # get similarity between input data and the train data
        similarity = __calc_similarity(transistion_matrix)
        # transit similarity to weights
        weights = __inversion_kernel(similarity)
        # predict the classification
        self._clf_res.append(__clf_predict(weights))
    
    @tf.function
    def __body(self, i, ipt):
        row = self._inputs[i]
        __WKNN(row)
        return (i, row)
    
    def call(self, inputs):
        # initialization test data
        self._inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        self.clf_res =tf.zeros(shape=(1,self._inputs.shape[1]))
        # call algorithm in a loop
        index = tf.constant(0)
        c = lambda i, ipt: tf.less(i, self._inputs.shape[1])
        _ = tf.while_loop(c, __body, [index, self._inputs[index]])
        # return result
        return self._clf_res

# Unsupervised Learning Model